In [ ]:
import csv
import random

class Noeud:
    def __init__(self, grille=None, joueur_actuel=1):
        if grille is None:
            self.grille = [
                [1, 1, 1],  # Joueur 1 (Noir) en haut
                [0, 0, 0],  # Ligne vide au centre
                [-1, -1, -1]  # Joueur 2 (Gris) en bas
            ]
        else:
            self.grille = [ligne[:] for ligne in grille]  # Copie du plateau
        self.joueur_actuel = joueur_actuel

    def afficher_grille(self):
        """Affiche le plateau sous forme lisible."""
        symbols = {0: ".", 1: "X", -1: "O"}
        for ligne in self.grille:
            print(" ".join(symbols[cell] for cell in ligne))
        print()

    def deplacements_possibles(self, x, y):
        """Retourne une liste des mouvements valides pour un pion donné."""
        directions = [
            (0, 1), (0, -1), (1, 0), (-1, 0),  # Déplacements horizontaux et verticaux
            (1, 1), (1, -1), (-1, 1), (-1, -1)  # Déplacements diagonaux
        ]
        mouvements = []
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 3 and 0 <= ny < 3 and self.grille[nx][ny] == 0:
                mouvements.append((nx, ny))
        return mouvements

    def get_successor(self):
        """Génère tous les états possibles après un coup."""
        successors = []
        for i in range(3):
            for j in range(3):
                if self.grille[i][j] == self.joueur_actuel:
                    for nx, ny in self.deplacements_possibles(i, j):
                        nouvelle_grille = [ligne[:] for ligne in self.grille]
                        nouvelle_grille[i][j] = 0
                        nouvelle_grille[nx][ny] = self.joueur_actuel
                        successors.append(Noeud(nouvelle_grille, -self.joueur_actuel))
        return successors

    def check_winner(self):
        """Vérifie si le joueur actuel a gagné."""
        for i in range(3):
            # Vérification horizontale et verticale
            if abs(sum(self.grille[i])) == 3 or abs(sum(row[i] for row in self.grille)) == 3:
                return True
        # Vérification diagonale
        if abs(self.grille[0][0] + self.grille[1][1] + self.grille[2][2]) == 3:
            return True
        if abs(self.grille[0][2] + self.grille[1][1] + self.grille[2][0]) == 3:
            return True
        return False

    def is_terminal(self):
        """Vérifie si le jeu est terminé."""
        return self.check_winner() or all(self.grille[i][j] != 0 for i in range(3) for j in range(3))

    def eval(self, us):
        """Évalue l'état actuel du jeu pour le joueur 'us'."""
        if self.check_winner():
            return 1 if self.joueur_actuel == us else -1
        elif all(self.grille[i][j] != 0 for i in range(3) for j in range(3)):
            return 0  # Match nul
        return 0  # Jeu en cours

def minimax(node, depth, us):
    """Algorithme Minimax pour choisir le meilleur coup."""
    if depth == 0 or node.is_terminal():
        return (node.eval(us), None)

    if node.joueur_actuel == us:
        maxEval = -float('inf')
        best_move = None
        for child in node.get_successor():
            eval, _ = minimax(child, depth - 1, us)
            if eval > maxEval:
                maxEval = eval
                best_move = child
        return (maxEval, best_move)
    else:
        minEval = float('inf')
        best_move = None
        for child in node.get_successor():
            eval, _ = minimax(child, depth - 1, us)
            if eval < minEval:
                minEval = eval
                best_move = child
        return (minEval, best_move)

def jouer_partie():
    """Joue une partie complète et retourne l'état final et le gagnant."""
    noeud_actuel = Noeud()
    while not noeud_actuel.is_terminal():
        _, meilleur_coup = minimax(noeud_actuel, depth=3, us=noeud_actuel.joueur_actuel)
        if meilleur_coup:
            noeud_actuel = meilleur_coup
        else:
            break  # Aucun coup possible (match nul)
    return noeud_actuel.grille, noeud_actuel.joueur_actuel if noeud_actuel.check_winner() else 0

def generer_victoires(nb_victoires, joueur):
    """Génère un nombre donné de victoires pour un joueur spécifique."""
    victoires = []
    while len(victoires) < nb_victoires:
        grille, gagnant = jouer_partie()
        if gagnant == joueur:
            victoires.append((grille, gagnant))
    return victoires

def sauvegarder_en_csv(victoires, filename):
    """Sauvegarde les victoires dans un fichier CSV."""
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Grille", "Gagnant"])
        for grille, gagnant in victoires:
            writer.writerow([str(grille), gagnant])

# Générer 500 victoires pour le joueur noir (1) et 500 pour le joueur gris (-1)
victoires_noir = generer_victoires(500, 1)
victoires_gris = generer_victoires(500, -1)

# Sauvegarder les résultats dans des fichiers CSV
sauvegarder_en_csv(victoires_noir, "victoires_noir.csv")
sauvegarder_en_csv(victoires_gris, "victoires_gris.csv")

print("500 victoires pour le joueur noir et 500 victoires pour le joueur gris ont été générées et sauvegardées.")